In [ ]:
#Create time series to feed into autoencoder

import numpy as np
import os
from sklearn.preprocessing import normalize
from pandas import read_csv
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical
from sklearn.metrics import classification_report
#Create 3d array
def create_time_series(filepath, n_timesteps):

    time_series = []
    Y = []
    first_y = True
    first_windows = True
    for filename in os.listdir(filepath):
        print(filename)
        data = np.load(filepath + "/" + filename)
        data = normalize(data, axis=1)
        windows = create_sliding_windows(data, n_timesteps)
        y_category = create_Y_for_category(filename, windows.shape[0])
        if first_y:
            Y = np.array(y_category)
            first_y = False
        else:
            Y = np.vstack((Y,y_category))

        if first_windows:
            time_series = windows
            first_windows = False
        else:
            time_series = np.vstack((time_series, windows))

    return time_series, Y



# Create windows of n_timesteps BY ADDING PADDING AND THEN SPLITTING THE ARRAY
def split_into_windows(X, n_timesteps):

    windows = []

    # Fill array until it can be split into windows of 10 timesteps (elements)
    while len(X) % n_timesteps != 0:
        X = np.row_stack((X, X[ len(X) -1 ]))

    windows = np.array_split(X, (len(X) / n_timesteps), axis=0)
    return np.array(windows)


#Create y data for one category
def create_Y_for_category(filename, windows_length):

    indexLabel = findIndexLabel(filename)
    Y = np.full((windows_length, 1), indexLabel)
    return Y


# Find index of the label assigned to the exercise
def findIndexLabel(filename):
    categories = load_txt("/content/drive/My Drive/Colab Notebooks/categories")
    for index in range(len(categories)):
        if filename.split(".")[0]  == categories[index]:
            return index


# load a single file txt as a numpy array
def load_txt(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values


# Create sliding windows of n_timesteps (not used)
def create_sliding_windows(data, n_timesteps):

    windows = []
    for i in range(data.shape[0] - n_timesteps):
        windows.append(data[i: i + n_timesteps])

    return np.array(windows)



#LSTM-Autoencoder Training

#Big function to extract features
def extract_features(filepath):
    n_timesteps = 60 #CHANGE TIMESTEPS HERE
    X, Y = create_time_series(filepath, n_timesteps)
    print(X.shape)
    print(Y.shape)
    X_encoded = evaluate_AE_model(X) #Evaluate model and extract features
    print(X_encoded.shape)
    return X_encoded, Y


# Create, train and evaluate model
def evaluate_AE_model(X):

    print("TRAINING LSTM AUTOENCODER")

    #plt.plt(X[0])
    n_timesteps = X.shape[1] 
    n_features = X.shape[2]
   
    #create encoder part
    encoder_decoder = Sequential()
    encoder_decoder.add(LSTM(128, activation='relu', input_shape=(n_timesteps, n_features), return_sequences=True))
    encoder_decoder.add(LSTM(64, activation='relu', input_shape=(n_timesteps, n_features), return_sequences=False))
    ## Le nombre de features extrait depend en fait du nombre de neurones du dernier layer de l encodeur 

    #create decoder part
    encoder_decoder.add(RepeatVector(n_timesteps))
    encoder_decoder.add(LSTM(64, activation='relu', return_sequences=True))
    encoder_decoder.add(LSTM(128, activation='relu', return_sequences=True))
    encoder_decoder.add(TimeDistributed(Dense(n_features)))

    n_epochs = 5
    encoder_decoder.compile(optimizer= "adam", loss='mse')
    encoder_decoder.fit(X, X, epochs=n_epochs,verbose=1, batch_size=100)

    encoder_decoder.save("/content/drive/My Drive/Colab Notebooks/LSTMAE_" + str(n_timesteps) + "timesteps_" + str(n_epochs) + "_sw" )
    # X_pred = encoder_decoder.predict(X)
    # plt.plot(X_pred[0])

    #Extract features
    encoder = Model(inputs=encoder_decoder.inputs, outputs=encoder_decoder.layers[1].output)
    X_encoded = encoder.predict(X)

    return X_encoded



# fit and evaluate DNN or RF
def evaluate_model(X_train, y_train, X_test, y_test):

    #Neural network

    print("TRAINING CLASSIFIER")

    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    n_epochs = 20
    model.fit(X_train, y_train, epochs=n_epochs, batch_size=100)

    _, accuracy = model.evaluate(X_test, y_test,verbose=1)

    y_pred = model.predict(X_test)

    model.save("/content/drive/My Drive/Colab Notebooks/DNN_" + str(n_epochs) + "_" + str(accuracy * 100) + "_sw")

    print('Accuracy: %.2f' % (accuracy*100))


    y_pred = np.argmax(y_pred,axis = 1) 
    y_test = np.argmax(y_test,axis = 1)
    print(y_pred.shape)
    print(y_test.shape)
    print(confusion_matrix(y_pred, y_test))
    print(classification_report(y_test, y_pred))
    # RANDOM FOREST
    # RF = RandomForestClassifier(n_estimators=250, max_depth=2,
    #                          random_state=3)
    # RF.fit(X_train, y_train)
    # score = RF.score(X_test, y_test)
    # print(score)
   
# main function to call
def run_experiment():
    file_name = "/content/drive/My Drive/Colab Notebooks/New Dataset"
    X_data, Y_data = extract_features(file_name)

    Y_data = to_categorical(Y_data)

    X_train, X_test, y_train, y_test = train_test_split(
        X_data, Y_data, stratify=Y_data,test_size=0.30, random_state=42
    )

    evaluate_model(X_train, y_train, X_test, y_test)



run_experiment()
   





HandOpen.npy
HandRest.npy
ObjectGrip.npy
PichGrip.npy
WristPron.npy
WristSupi.npy
WristExten.npy
WristFlex.npy
(1659520, 60, 7)
(1659520, 1)
TRAINING LSTM AUTOENCODER
Epoch 1/5
